# Qubit spectroscopy: Rabi measurement example
This notebook demonstrates how to perform a Rabi experiment on a simulated or real quantum device. This experiment tests the calibration of the `RX` pulse by rotating through a full $2\pi$ radians and evaluating the excited state visibility as a function of the angle of rotation, $\theta$. The QUIL program for one data point for qubit 0 at, for example $\theta=\pi/2$, is

    DECLARE ro BIT[1]
    RX(pi/2) 0
    MEASURE 0 ro[0]
   
We expect to see a characteristic "Rabi flop" by sweeping $\theta$ over $[0, 2\pi)$, thereby completing a full rotation around the Bloch sphere. It should look like $\dfrac{1-\cos(\theta)}{2}$

## setup

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from pyquil.api import get_qc, QuantumComputer

from forest.benchmarking.qubit_spectroscopy import *

## measurement, analysis, and plotting

This method takes a `pandas.DataFrame` object with all the Rabi data and plots the Rabi flop. It fits to a sinusoid and evaluates the period for each qubit (which should be $2\pi$). 

In [ ]:
#qc = get_qc('Aspen-1-15Q-A')
qc = get_qc('2q-noisy-qvm') # will run on a QVM, but not meaningfully 
qubits = qc.qubits()
qubits

**Generate simultaneous Rabi experiments**


In [ ]:
import numpy as np
from numpy import pi
angles = np.linspace(0, 2*pi, 15)
rabi_expts = [generate_rabi_experiment(qubit, angles) for qubit in qubits]

In [ ]:
print(rabi_expts[0])

**Acquire data**

Collect our Rabi raw data using `acquire_rabi_data`. 

In [ ]:
num_shots = 500
acquire_rabi_data(qc,rabi_expts, num_shots)

**Use the results to produce estimates of Rabi curve**

In [ ]:
params1, errs1 = estimate_rabi(rabi_expts[0])
params2, errs2 = estimate_rabi(rabi_expts[1])
params = [params1, params2]
errs = [errs1, errs2]

**Now plot a subset with the fits**

In [ ]:
plot_rabi_estimate_over_data(rabi_expts, params, errs)

In [ ]:
print(params[0][3])